In [44]:
import os

import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
import warnings
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import *
from statsmodels.stats.multicomp import pairwise_tukeyhsd

warnings.filterwarnings("ignore")

In [45]:
file_paths = [f"GT_error/hirasaki_{i}_errors.csv" for i in range(10)]
df = pd.concat([pd.read_csv(file_path, index_col=0) for file_path in file_paths], ignore_index=True)

In [46]:
raw_combination_data = []
for col in df.columns:
    if "_mpjpe" in col or "_angle" in col:
        error_type = "MPJPE" if "_mpjpe" in col else col.split('_')[1] + " " + col.split('_')[2]
        if len(col.split('_')) == 6:
            error_type += " " + col.split('_')[3]
        for value in df[col].values:
            raw_combination_data.append({"Combination": col.split('_')[0], "Error Type": error_type, "Error Value": value})

raw_combination_df = pd.DataFrame(raw_combination_data)
raw_combination_df.head()

,Combination,Error Type,Error Value
0,01,MPJPE,15.95
1,01,MPJPE,15.69
2,01,MPJPE,15.46
3,01,MPJPE,15.26
4,01,MPJPE,15.04


In [47]:
raw_combination_df = raw_combination_df[raw_combination_df["Combination"] != "04"]
raw_combination_df = raw_combination_df[raw_combination_df["Error Type"] != "MPJPE"]

In [48]:
raw_combination_df["camera_num"] = raw_combination_df["Combination"].apply(lambda x: len(x))

In [49]:
camera_2_combinations = raw_combination_df[raw_combination_df["camera_num"] == 2]
camera_3_combinations = raw_combination_df[raw_combination_df["camera_num"] == 3]
camera_4_combinations = raw_combination_df[raw_combination_df["camera_num"] == 4]
camera_5_combinations = raw_combination_df[raw_combination_df["camera_num"] == 5]

In [50]:
def calc_threshold_ratios(df, thresholds):
    result = []
    for threshold in thresholds:
        temp = df.groupby("Error Type")["Error Value"].apply(lambda x: (x < threshold).sum() / len(x)).reset_index()
        temp["Threshold"] = threshold
        temp.rename(columns={"Error Value": "Ratio"}, inplace=True)
        result.append(temp)
    
    final_df = pd.concat(result, ignore_index=True)
    pivot_df = final_df.pivot(index="Error Type", columns="Threshold", values="Ratio")
    pivot_df.reset_index(inplace=True)
    return pivot_df

# Example usage
thresholds = [1, 1.5, 2, 2.5, 3]  # List of thresholds to evaluate
print("2 cameras")
display(calc_threshold_ratios(camera_2_combinations, thresholds))
print("3 cameras")
display(calc_threshold_ratios(camera_3_combinations, thresholds))
print("4 cameras")
display(calc_threshold_ratios(camera_4_combinations, thresholds))
print("5 cameras")
display(calc_threshold_ratios(camera_5_combinations, thresholds))


2 cameras


Threshold,Error Type,1.0,1.5,2.0,2.5,3.0
0,LEFT ELBOW,0.81,0.88,0.92,0.95,0.96
1,LEFT KNEE,0.78,0.87,0.91,0.94,0.96
2,LEFT KNEE UP,0.76,0.86,0.91,0.93,0.94
3,RIGHT ELBOW,0.69,0.82,0.89,0.91,0.93
4,RIGHT KNEE,0.75,0.84,0.90,0.94,0.95
5,RIGHT KNEE UP,0.73,0.82,0.88,0.94,0.96


3 cameras


Threshold,Error Type,1.0,1.5,2.0,2.5,3.0
0,LEFT ELBOW,0.87,0.94,0.98,0.99,0.99
1,LEFT KNEE,0.91,0.95,0.98,0.99,1.00
2,LEFT KNEE UP,0.86,0.94,0.98,0.99,1.00
3,RIGHT ELBOW,0.83,0.90,0.95,0.97,0.99
4,RIGHT KNEE,0.90,0.95,0.98,0.99,1.00
5,RIGHT KNEE UP,0.82,0.93,0.97,0.99,0.99


4 cameras


Threshold,Error Type,1.0,1.5,2.0,2.5,3.0
0,LEFT ELBOW,0.94,0.99,1.00,1.00,1.00
1,LEFT KNEE,0.96,0.99,1.00,1.00,1.00
2,LEFT KNEE UP,0.95,0.98,1.00,1.00,1.00
3,RIGHT ELBOW,0.87,0.96,0.98,0.99,1.00
4,RIGHT KNEE,0.97,1.00,1.00,1.00,1.00
5,RIGHT KNEE UP,0.89,0.99,0.99,1.00,1.00


5 cameras


Threshold,Error Type,1.0,1.5,2.0,2.5,3.0
0,LEFT ELBOW,0.99,1.00,1.00,1.00,1.00
1,LEFT KNEE,1.00,1.00,1.00,1.00,1.00
2,LEFT KNEE UP,1.00,1.00,1.00,1.00,1.00
3,RIGHT ELBOW,0.88,1.00,1.00,1.00,1.00
4,RIGHT KNEE,1.00,1.00,1.00,1.00,1.00
5,RIGHT KNEE UP,0.99,1.00,1.00,1.00,1.00
